In [1]:
# import here
import pandas as pd
from sqlalchemy import create_engine
from references_dict import Team_Dictionary

# connect to database
engine = create_engine('mysql+pymysql://root:@localhost:3306/nfl_db')
conn = engine.connect()
file = ("D:\\NFLDB\\game_info.csv")

In [2]:
# trim csv file to relevant stats for weeks 1-16, 2009-2018
df = pd.read_csv(file)

# drop playoff weeks
indexNames = df[ df['schedule_playoff'] == True ].index
df.drop(indexNames,inplace=True)

# drop stats older than 2009
indexNames = df[ df['schedule_season'] < 2009 ].index
df.drop(indexNames,inplace=True)

In [3]:
def get_home_favorite(row):
	home_team = row['team_home']
	home_abbrev = Team_Dictionary().kaggle_games_abbrev[home_team]
	if home_abbrev == row['team_favorite_id']:
		return 1
	else:
		return 0
    
def get_spread_result(row):
    score_fav = 0
    score_und = 0
    spread = row['spread_favorite']*-1
    if(row['home_favorite']==1):
        score_fav = row['score_home']
        score_und = row['score_away']
    else:
        score_fav = row['score_away']
        score_und = row['score_home']
    diff = score_fav-score_und
    if( diff > spread ):
        return 1
    elif( diff < spread ):
        return -1
    else:
        return 0
    
def get_OU_result(row):
    OU = float(row['over_under_line'])
    total = row['score_home']+row['score_away']
    if( total > OU ):
        return 1
    elif( total < OU ):
        return -1
    else:
        return 0

# def get_index(row):
#     season = row['schedule_season']
#     week = row['schedule_week']
#     home_team = Team_Dictionary().kaggle_games_abbrev[row['team_home']]
#     return str(season)+str(week).zfill(2)+home_team
def get_index(row):
    date = row['schedule_date']
    comps = date.split('/')
    date = comps[2]+'-'+comps[0]+'-'+comps[1]
    return date+Team_Dictionary().kaggle_games_abbrev[row['team_home']]

In [4]:
# # generate metrics for dataset, set index
df['home_favorite'] = df.apply (lambda row: get_home_favorite(row), axis=1)
df['spread_result'] = df.apply(lambda row: get_spread_result(row),axis=1)
df['OU_result'] = df.apply(lambda row: get_OU_result(row),axis=1)
df['idx'] = df.apply(lambda row: get_index(row),axis=1)
df.set_index('idx',inplace=True)

In [5]:
def get_pbpindex(row):
    team_dict = dict([[v,k] for k,v in Team_Dictionary().kaggle_plays_abbrev.items()])
    team_name = team_dict[row['home_team']]
    team_name = Team_Dictionary().kaggle_games_abbrev[team_name]
    idx = row['game_date']+team_name
    return idx

# sql statement for getting gameids
sql = "select distinct(pbp.game_id) as game_id, pbp.home_team as home_team, pbp.game_date as game_date \
       from nfl_pbp pbp \
       order by pbp.game_id"
df_gameids = pd.read_sql_query(sql, conn, index_col=None)
df_gameids['idx'] = df_gameids.apply(lambda row: get_pbpindex(row),axis=1)
df_gameids.drop(['home_team','game_date'],axis=1,inplace=True)
df_gameids.set_index('idx',inplace=True)

In [6]:
df['game_id']=df_gameids['game_id']
df.set_index('game_id')
df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,...,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,home_favorite,spread_result,OU_result,game_id
idx,,,,,,,,,,,,,,,,,,,,,
2009-09-10PIT,09/10/2009,2009,1,False,Pittsburgh Steelers,13,10,Tennessee Titans,PIT,-6.5,...,Heinz Field,False,67.0,9.0,64,NaN,1,-1,-1,2.009091e+09
2009-09-13ARI,09/13/2009,2009,1,False,Arizona Cardinals,16,20,San Francisco 49ers,ARI,-4.5,...,University of Phoenix Stadium,False,72.0,0.0,NaN,DOME,1,-1,-1,2.009091e+09
2009-09-13ATL,09/13/2009,2009,1,False,Atlanta Falcons,19,7,Miami Dolphins,ATL,-4.0,...,Georgia Dome,False,72.0,0.0,NaN,DOME,1,1,-1,2.009091e+09
2009-09-13BAL,09/13/2009,2009,1,False,Baltimore Ravens,38,24,Kansas City Chiefs,BAL,-13.0,...,M&T Bank Stadium,False,69.0,7.0,74,NaN,1,1,1,2.009091e+09
2009-09-13CAR,09/13/2009,2009,1,False,Carolina Panthers,10,38,Philadelphia Eagles,PHI,-2.5,...,Bank of America Stadium,False,77.0,7.0,59,NaN,0,1,1,2.009091e+09


In [7]:
# home stats
home_rush_sql = "select game_id, posteam as home_abbrev, sum(yards_gained) as home_rush_yds, sum(rush_attempt) as home_rush_att,sum(rush_touchdown) as home_rush_tds \
  from nfl_pbp \
  where posteam = home_team \
  and play_type = 'run' \
group by game_id"

home_short_pass_sql = "select game_id,sum(yards_gained) as home_short_pass_yds \
  from nfl_pbp \
  where play_type='pass' \
  and pass_length = 'short' \
  and posteam=home_team \
  group by game_id,posteam"

home_deep_pass_sql = "select game_id,sum(yards_gained) as home_deep_pass_yds \
  from nfl_pbp \
  where play_type='pass' \
  and pass_length = 'deep' \
  and posteam=home_team \
  group by game_id,posteam"

home_pass_total_sql = "select game_id, \
       sum(pass_attempt) as home_pass_att, \
       sum(complete_pass) as home_completions, \
       sum(pass_touchdown) as home_pass_tds, \
       sum(sack) as home_sacked, \
       sum(interception) as home_interceptions \
  from nfl_pbp \
  where play_type = 'pass' \
  and posteam = home_team \
  group by game_id"

home_rush_mets = pd.read_sql_query(home_rush_sql, conn, index_col=None)
home_rush_mets.set_index('game_id',inplace=True)
home_short_pass_mets = pd.read_sql_query(home_short_pass_sql, conn, index_col=None)
home_short_pass_mets.set_index('game_id',inplace=True)
home_deep_pass_mets = pd.read_sql_query(home_deep_pass_sql, conn, index_col=None)
home_deep_pass_mets.set_index('game_id',inplace=True)
home_pass_mets = pd.read_sql_query(home_pass_total_sql, conn, index_col=None)
home_pass_mets.set_index('game_id',inplace=True)

home_offense = home_rush_mets.merge(home_short_pass_mets,on='game_id')
home_offense = home_offense.merge(home_deep_pass_mets,on='game_id')
home_offense = home_offense.merge(home_pass_mets,on='game_id')

home_offense.head()

,home_abbrev,home_rush_yds,home_rush_att,home_rush_tds,home_short_pass_yds,home_deep_pass_yds,home_pass_att,home_completions,home_pass_tds,home_sacked,home_interceptions
game_id,,,,,,,,,,,
2009091000,PIT,36.0,23.0,0.0,218.0,145.0,47.0,33.0,1.0,4.0,2.0
2009091300,ATL,72.0,24.0,0.0,207.0,22.0,38.0,22.0,2.0,2.0,0.0
2009091301,BAL,198.0,41.0,2.0,195.0,112.0,44.0,26.0,3.0,1.0,1.0
2009091302,CAR,87.0,29.0,1.0,94.0,39.0,39.0,14.0,0.0,5.0,5.0
2009091303,CIN,86.0,27.0,1.0,247.0,0.0,36.0,21.0,0.0,3.0,2.0


In [8]:
# home stats
away_rush_sql = "select game_id, \
    posteam as away_abbrev, \
    sum(yards_gained) as away_rush_yds, \
    sum(rush_attempt) as away_rush_att, \
    sum(rush_touchdown) as away_rush_tds \
  from nfl_pbp \
  where posteam = away_team \
  and play_type = 'run' \
group by game_id"

away_short_pass_sql = "select game_id,sum(yards_gained) as away_short_pass_yds \
  from nfl_pbp \
  where play_type='pass' \
  and pass_length = 'short' \
  and posteam = away_team \
  group by game_id,posteam"

away_deep_pass_sql = "select game_id,sum(yards_gained) as away_deep_pass_yds \
  from nfl_pbp \
  where play_type='pass' \
  and pass_length = 'deep' \
  and posteam = away_team \
  group by game_id,posteam"

away_pass_total_sql = "select game_id, \
       sum(pass_attempt) as away_pass_att, \
       sum(complete_pass) as away_completions, \
       sum(pass_touchdown) as away_pass_tds, \
       sum(sack) as away_sacked, \
       sum(interception) as away_interceptions \
  from nfl_pbp \
  where play_type = 'pass' \
  and posteam = away_team \
  group by game_id"

away_rush_mets = pd.read_sql_query(away_rush_sql, conn, index_col=None)
away_rush_mets.set_index('game_id',inplace=True)
away_short_pass_mets = pd.read_sql_query(away_short_pass_sql, conn, index_col=None)
away_short_pass_mets.set_index('game_id',inplace=True)
away_deep_pass_mets = pd.read_sql_query(away_deep_pass_sql, conn, index_col=None)
away_deep_pass_mets.set_index('game_id',inplace=True)
away_pass_mets = pd.read_sql_query(away_pass_total_sql, conn, index_col=None)
away_pass_mets.set_index('game_id',inplace=True)

away_offense = away_rush_mets.merge(away_short_pass_mets,on='game_id')
away_offense = away_offense.merge(away_deep_pass_mets,on='game_id')
away_offense = away_offense.merge(away_pass_mets,on='game_id')

away_offense.head()

,away_abbrev,away_rush_yds,away_rush_att,away_rush_tds,away_short_pass_yds,away_deep_pass_yds,away_pass_att,away_completions,away_pass_tds,away_sacked,away_interceptions
game_id,,,,,,,,,,,
2009091000,TEN,86.0,25.0,0.0,152.0,92.0,36.0,22.0,1.0,1.0,1.0
2009091300,MIA,96.0,22.0,0.0,139.0,37.0,33.0,21.0,1.0,4.0,1.0
2009091301,KC,29.0,17.0,0.0,127.0,50.0,27.0,16.0,2.0,3.0,0.0
2009091302,PHI,186.0,31.0,1.0,102.0,0.0,31.0,17.0,2.0,2.0,1.0
2009091303,DEN,75.0,20.0,0.0,106.0,137.0,32.0,17.0,1.0,3.0,0.0


In [9]:
all_offense = home_offense.merge(away_offense,on='game_id')
all_offense.head()

,home_abbrev,home_rush_yds,home_rush_att,home_rush_tds,home_short_pass_yds,home_deep_pass_yds,home_pass_att,home_completions,home_pass_tds,home_sacked,...,away_rush_yds,away_rush_att,away_rush_tds,away_short_pass_yds,away_deep_pass_yds,away_pass_att,away_completions,away_pass_tds,away_sacked,away_interceptions
game_id,,,,,,,,,,,,,,,,,,,,,
2009091000,PIT,36.0,23.0,0.0,218.0,145.0,47.0,33.0,1.0,4.0,...,86.0,25.0,0.0,152.0,92.0,36.0,22.0,1.0,1.0,1.0
2009091300,ATL,72.0,24.0,0.0,207.0,22.0,38.0,22.0,2.0,2.0,...,96.0,22.0,0.0,139.0,37.0,33.0,21.0,1.0,4.0,1.0
2009091301,BAL,198.0,41.0,2.0,195.0,112.0,44.0,26.0,3.0,1.0,...,29.0,17.0,0.0,127.0,50.0,27.0,16.0,2.0,3.0,0.0
2009091302,CAR,87.0,29.0,1.0,94.0,39.0,39.0,14.0,0.0,5.0,...,186.0,31.0,1.0,102.0,0.0,31.0,17.0,2.0,2.0,1.0
2009091303,CIN,86.0,27.0,1.0,247.0,0.0,36.0,21.0,0.0,3.0,...,75.0,20.0,0.0,106.0,137.0,32.0,17.0,1.0,3.0,0.0


In [10]:
df = df.merge(all_offense,on='game_id')
df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,...,away_rush_yds,away_rush_att,away_rush_tds,away_short_pass_yds,away_deep_pass_yds,away_pass_att,away_completions,away_pass_tds,away_sacked,away_interceptions
0,09/10/2009,2009,1,False,Pittsburgh Steelers,13,10,Tennessee Titans,PIT,-6.5,...,86.0,25.0,0.0,152.0,92.0,36.0,22.0,1.0,1.0,1.0
1,09/13/2009,2009,1,False,Arizona Cardinals,16,20,San Francisco 49ers,ARI,-4.5,...,21.0,25.0,1.0,159.0,50.0,35.0,18.0,1.0,4.0,0.0
2,09/13/2009,2009,1,False,Atlanta Falcons,19,7,Miami Dolphins,ATL,-4.0,...,96.0,22.0,0.0,139.0,37.0,33.0,21.0,1.0,4.0,1.0
3,09/13/2009,2009,1,False,Baltimore Ravens,38,24,Kansas City Chiefs,BAL,-13.0,...,29.0,17.0,0.0,127.0,50.0,27.0,16.0,2.0,3.0,0.0
4,09/13/2009,2009,1,False,Carolina Panthers,10,38,Philadelphia Eagles,PHI,-2.5,...,186.0,31.0,1.0,102.0,0.0,31.0,17.0,2.0,2.0,1.0


In [11]:
def calculate_home_rushFP(row):
    yards = row['home_rush_yds']
    ypa = yards/row['home_rush_att']
    tds = row['home_rush_tds']
    points = (yards/10)+(tds*6)+ypa
    return points

def calculate_away_rushFP(row):
    yards = row['away_rush_yds']
    ypa = yards/row['away_rush_att']
    tds = row['away_rush_tds']
    points = (yards/10)+(tds*6)+ypa
    return points

def calculate_home_passFP(row):
    yards = row['home_short_pass_yds']+row['home_deep_pass_yds']
    tds = row['home_pass_tds']
    completion_pct = row['home_completions']/row['home_pass_att']
    points = (yards/10)+(tds*6)+(row['home_completions']*completion_pct)-(row['home_interceptions']*2)-row['home_sacked']
    return points

def calculate_away_passFP(row):
    yards = row['away_short_pass_yds']+row['away_deep_pass_yds']
    tds = row['away_pass_tds']
    completion_pct = row['away_completions']/row['away_pass_att']
    points = (yards/10)+(tds*6)+(row['away_completions']*completion_pct)-(row['away_interceptions']*2)-row['away_sacked']
    return points

In [12]:
df['rush_skillpoints_home'] = df.apply (lambda row: calculate_home_rushFP(row), axis=1)
df['rush_skillpoints_away'] = df.apply (lambda row: calculate_away_rushFP(row), axis=1)
df['pass_skillpoints_home'] = df.apply (lambda row: calculate_home_passFP(row), axis=1)
df['pass_skillpoints_away'] = df.apply (lambda row: calculate_away_passFP(row), axis=1)

In [13]:
df.to_sql('nfl_gameinfo', con=engine, if_exists='replace',index=False)